In [31]:
# Library Import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.utils.data as data_utils
import random
import tqdm
import numpy as np
import sklearn
import pandas as pd
import os
import skimage.measure
import plyfile
import os
torch.cuda.empty_cache()
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import gc
gc.collect()
torch.cuda.empty_cache()
import time

In [32]:
random.seed(31359)
torch.random.manual_seed(31359)
np.random.seed(31359)
!nvidia-smi

Mon Jun  6 10:49:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 52%   26C    P8    23W / 350W |      1MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:25:00.0 Off |                  N/A |
| 52%   

In [33]:
DIRECTORY_MODELS = 'Dataset/04256520/'  # 04256520: Sofa
#DIRECTORY_MODELS = 'Dataset/02691156' # 02691156 : Plane
#DIRECTORY_MODELS = 'Dataset/04379243' # 04379243: Table

MODEL_EXTENSION = '.npz'
def get_model_files():
    for directory, _, files in os.walk(DIRECTORY_MODELS):
        for filename in files:
            if filename.endswith(MODEL_EXTENSION):
                yield  os.path.join(filename)
    
def get_model_dir():
    for directory, _, files in os.walk(DIRECTORY_MODELS):
        for filename in files:
            if filename.endswith(MODEL_EXTENSION):
                yield os.path.join(directory, filename)
                
files = list(get_model_files())
dirs = list(get_model_dir())

print("the number of Dataset:", len(files))


the number of Dataset: 8


In [34]:
num_train = 1000
num_test = 100
# train set!
files_dir_test = dirs[:num_test] # /data.../...npz
files_npz_test = files[:num_test] # ...npz
len(files_npz_train)
files_dir_train = dirs[num_test+1:num_train+num_test+1] # /data.../...npz
files_npz_train = files[num_test+10:num_train+num_test+10] # ...npz
print("the number of Dataset(train):", len(files_npz_train))
print("the number of Dataset(test):", len(files_npz_test))# .npz

files_npz_test = sorted(files_npz_test)


the number of Dataset(train): 0
the number of Dataset(test): 8


In [35]:
# From the DIT github
# https://github.com/ZhengZerong/DeepImplicitTemplates

#weight initialization 
def weight_initial(self):
    for m in self.modules():
        for name, param in m.named_parameters():
            #print(name)
           # print(param)
            if 'weight_ih' in name:
                nn.init.kaiming_normal_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
            #print(param)
            
#output weight initialization
def weights_out_init(self):
    for m in self.modules():
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.uniform_(param.data, -1e-5, 1e-5)
            elif 'bias' in name:
                nn.init.constant_(param.data, 0)

In [36]:
# output: warped point (from warper) and predicted SDF (from MLP)

class DIT(nn.Module):
    def __init__(self, mlp_args, lstm_args):
        """
            the entire network consisting of the LSTM and the MLP
        """
        super(DIT, self).__init__()
        self.mlp = MLP(**mlp_args)
        self.lstm = Warper(**lstm_args)
      
    def forward(self, x):
        # get warped points
        p_canonical, warping_param, intermediate_xyzs = self.lstm(x)

        if self.training:
            # get sdf value for each intermediate warping step
            sdf_values = []
            for points in intermediate_xyzs:#step:
                sdf = self.mlp(torch.tensor(points, device = 'cuda'))
                sdf_values.append(torch.tensor(sdf, device = 'cuda').squeeze())
            return sdf_values, intermediate_xyzs
        else:
       #     print("No Train")
             # only final sdf value
            sdf = self.mlp(p_canonical)
            #squeeze된 sdf
            return sdf, p_canonical

    

In [37]:

class Warper(nn.Module):
    def __init__(
            self,
            steps,
            latent_size,
            h_size,
            dim,
            linee = False
    ):
        super(Warper, self).__init__()
        #latent vector + xyz coordinates
        out_len = 6
        self.n_feature_channels = latent_size + 3
        self.h_size = h_size
        self.steps = steps
        self.lstm = nn.LSTMCell(input_size=self.n_feature_channels,
                                hidden_size=h_size)
        lstm_layer = self.lstm
        out_len = 6

        #weight initial
        lstm_layer.apply(weight_initial)
        for n, p in lstm_layer.named_parameters():
            if "bias" in n: # continue
                si = p.size(0)
                start, end = si // 4, si // 2
                p.data[start:end].fill_(1.)

        self.out_feature = nn.Linear(h_size, out_len)
        #out weight initializer
        self.out_feature.apply(weights_out_init)

    def forward(self, input):
        xyz = input[:, -3:] #change
        lat = input[:, :-3] # fix
        states = [None]
        warped_xyzs = []
        warping_p = []

        # Step for 8 times
        for s in range(self.steps):
            cell_st = self.lstm( torch.cat([lat, xyz], dim=1), states[-1]) #Latest state!
            rgd = cell_st[0].requires_grad
            if rgd:
                cell_st[0].register_hook(lambda x: x.clamp(min=-10, max=10))
            a = self.out_feature(cell_st[0])
            
            int_xyz = torch.addcmul(a[:, 3:], (a[:, :3]+1), xyz)
                                    #SDF + 1*(1+ w_xyz)*xyz
            
            if (s+1) % 2 == 0:
                warped_xyzs.append(int_xyz)
                
            states.append(cell_st)
            warping_p.append(a)

                #중간에 있는 값들!
            xyz = int_xyz
        return xyz, warping_p, warped_xyzs 
        #latest xyz, parameter, intermediate xyzs

In [38]:
# MLP : SDF Decoder

class MLP(nn.Module):
    def __init__(self, layers, wn_layers, weight_norm, dropout_layers, dropout_prob):
        """
            Initialize the MLP that extracts the template SDF
        """
        
        # layers: number of layers + size of each layer
        # [256, 256, 256, 256, 256] 
        # => 5 layers with 256 neurons each
        
        # wn_layers: layer indices in which normalization is used
        # [0, 1, 2, 3, 4]
        
        # weight_norm: bool
        # which normalization to use????? TODO
        
        # dropout_layers: layer indices in which dropout is used
        # [0, 1, 2, 3, 4]
        
        # dropout_prob: probability for dropout
        # 0.05
        
        super(MLP, self).__init__()
        
        
        self.numlayers = len(layers)+1
        #print("numlayers", self.numlayers)
        
        in_dim = 3
        out_dim = 1
        
        self.layers = nn.ModuleList()
        for i in range(self.numlayers):
            #print(i)
            # layer input feature count (for first layer: in_dim)
            in_features = in_dim if (i == 0) else layers[i-1]
            # layer output feature count (for last layer: out_dim)
            out_features = out_dim if (i == (len(layers))) else layers[i]
            
            # fully connected layer
            layer = nn.Linear(in_features, out_features)
            
            modules = [layer]
            if i in wn_layers:
                # weight normalization layer
                if False:  ##########weight_norm: ########## TODO
                    layer = nn.utils.weight_norm(layer) # ????????
                    modules[0] = layer
                else:
                    modules.append(nn.LayerNorm(out_features))
                    modules.append(nn.BatchNorm1d(out_features))
        
            # activation (tanh if last layer, else relu)
            activation = nn.Tanh() if (i == (len(layers))) else nn.ReLU()
            modules.append(activation)
            
            # dropout
            if i in dropout_layers: ### TODO only if training??
                modules.append(nn.Dropout(dropout_prob))
            

            sequential = nn.Sequential(*modules)
            self.layers.append(sequential)
            
    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
        return x # .squeeze()

In [43]:
#Code from the DeepSDF GitHub
#https://github.com/facebookresearch/DeepSDF/blob/48c19b8d49ed5293da4edd7da8c3941444bc5cd7/deep_sdf/data.py

repeat = 1
save_latvec_only = False #True면 Mesh 파일은 저장 X

rec_dir = 'reconstruction/'
data = DIRECTORY_MODELS.split('/')
recon_dir = rec_dir + data[1]
mesh_dir = '/mesh/'
lat_dir = '/latent/'


if data[1] == "04256520":
    class1 = 'weights_sofa'
elif data[1] == "04379243":
    class1 = 'weights_table'

elif data[1] == "02691156":
    class1 = 'weights_plane'
    
    
if not os.path.exists(rec_dir):
    os.mkdir(rec_dir)
if not os.path.exists(recon_dir):
    os.mkdir(recon_dir)
if not os.path.exists(recon_dir+mesh_dir):
    os.mkdir(recon_dir+mesh_dir)
mesh_dd = recon_dir+mesh_dir
lat_dd  = recon_dir+lat_dir
if not os.path.exists(recon_dir+lat_dir):
    os.mkdir(recon_dir+lat_dir)
    
       

def unpack(data, subsample=None):
    if subsample is None:
        return data
    pos_tensor = data[0]
    neg_tensor = data[1]

    # split the sample into half
    half = int(subsample / 2)

    pos_size = pos_tensor.shape[0]
    neg_size = neg_tensor.shape[0]

    pos_start_ind = random.randint(0, pos_size - half)
    sample_pos = pos_tensor[pos_start_ind : (pos_start_ind + half)]

    if neg_size <= half:
        random_neg = (torch.rand(half) * neg_tensor.shape[0]).long()
        sample_neg = torch.index_select(neg_tensor, 0, random_neg)
    else:
        neg_start_ind = random.randint(0, neg_size - half)
        sample_neg = neg_tensor[neg_start_ind : (neg_start_ind + half)]

    samples = torch.cat([sample_pos, sample_neg], 0)
   # print("samples: ",samples)
    randidx = torch.randperm(samples.shape[0])
    samples = torch.index_select(samples, 0, randidx)
 #   print("samples(OUT): ",samples)
    return samples

def create_mesh(
    decoder, latent_vec, filename, N=256, max_batch=(2**20), offset=None, scale=None, volume_size=2.0
    #decoder, latent_vec, filename, N=256, max_batch=(32 ** 3 * 4), offset=None, scale=None, volume_size=2.0
):
    start = time.time()
    ply_filename = filename

    # NOTE: the voxel_origin is actually the (bottom, left, down) corner, not the middle
    voxel_origin = [-volume_size/2.0, -volume_size/2.0, -volume_size/2.0]
#    voxel_origin = [-1.0,-1.0,-1.0]

    voxel_size = volume_size / (N - 1)

    overall_index = torch.arange(0, N ** 3, 1, out=torch.LongTensor())
    samples = torch.zeros(N ** 3, 4)

    # transform first 3 columns
    # to be the x, y, z index
    samples[:, 2] = overall_index % N
    samples[:, 1] = (overall_index.long() // N) % N
    samples[:, 0] = ((overall_index.long() // N) // N) % N

    # transform first 3 columns
    # to be the x, y, z coordinate
    samples[:, 0] = (samples[:, 0] * voxel_size) + voxel_origin[2]
    samples[:, 1] = (samples[:, 1] * voxel_size) + voxel_origin[1]
    samples[:, 2] = (samples[:, 2] * voxel_size) + voxel_origin[0]

    num_samples = N ** 3
    samples.requires_grad = False

    head = 0

    while head < num_samples:
        sample_subset = samples[head : min(head + max_batch, num_samples), 0:3].cuda()
        
        samples[head : min(head + max_batch, num_samples), 3] = (
            decode_sdf(decoder, latent_vec, sample_subset)
            .squeeze()
            .detach()
            .cuda()
        )
        head += max_batch
    sdf_values = samples[:, 3]
    print("sdf_values", sdf_values.min(), sdf_values.max())
    sdf_values = sdf_values.reshape(N, N, N)

    end = time.time()

    convert_sdf_samples_to_ply(
        sdf_values.data.cuda(),
        voxel_origin,
        voxel_size,
        ply_filename + ".ply",
        offset,
        scale,
    )



def decode_sdf(decoder, latent_vector, queries):
    num_samples = queries.shape[0]

    if latent_vector is None:
        inputs = queries
        sdf = decoder(inputs)[:, :1]
    else:
        try:
            latent_repeat = latent_vector.expand(num_samples, -1)
            inputs = torch.cat([latent_repeat, queries], 1)
            with torch.no_grad():
                sdf, p_can = decoder(inputs)

        except:
            raise RuntimeError("Failed to decode SDF")

    return sdf

def convert_sdf_samples_to_ply(
    input_3d_sdf_array,
    voxel_grid_origin,
    voxel_size,
    ply_filename_out,
    offset=None,
    scale=None,
):
    """
    Convert sdf samples to .ply

    :param input_3d_sdf_array: a float array of shape (n,n,n)
    :voxel_grid_origin: a list of three floats: the bottom, left, down origin of the voxel grid
    :voxel_size: float, the size of the voxels
    :ply_filename_out: string, path of the filename to save to

    This function adapted from: https://github.com/RobotLocomotion/spartan
    """
    start_time = time.time()

    if isinstance(input_3d_sdf_array, torch.Tensor):
        numpy_3d_sdf_tensor = input_3d_sdf_array.cpu().numpy()
    elif isinstance(input_3d_sdf_array, np.ndarray):
        numpy_3d_sdf_tensor = input_3d_sdf_array
    else:
        raise NotImplementedError
    print("numpy_3d_sdf_tensor",np.shape(numpy_3d_sdf_tensor)) #256 256 256
    print("numpy_3d_sdf_tensor",numpy_3d_sdf_tensor.min())
    print("numpy_3d_sdf_tensor",numpy_3d_sdf_tensor.max())
    
    verts, faces, normals, values = skimage.measure.marching_cubes(
        numpy_3d_sdf_tensor, level=0.0, spacing=[voxel_size] * 3,method ='lewiner'
    )

    # transform from voxel coordinates to camera coordinates
    # note x and y are flipped in the output of marching_cubes
    mesh_points = np.zeros_like(verts)
    mesh_points[:, 0] = voxel_grid_origin[0] + verts[:, 0]
    mesh_points[:, 1] = voxel_grid_origin[1] + verts[:, 1]
    mesh_points[:, 2] = voxel_grid_origin[2] + verts[:, 2]

    # apply additional offset and scale
    if scale is not None:
        mesh_points = mesh_points / scale
    if offset is not None:
        mesh_points = mesh_points - offset

    # try writing to the ply file

    num_verts = verts.shape[0]
    num_faces = faces.shape[0]

    verts_tuple = np.zeros((num_verts,), dtype=[("x", "f4"), ("y", "f4"), ("z", "f4")])

    for i in range(0, num_verts):
        verts_tuple[i] = tuple(mesh_points[i, :])

    faces_building = []
    for i in range(0, num_faces):
        faces_building.append(((faces[i, :].tolist(),)))
    faces_tuple = np.array(faces_building, dtype=[("vertex_indices", "i4", (3,))])

    el_verts = plyfile.PlyElement.describe(verts_tuple, "vertex")
    el_faces = plyfile.PlyElement.describe(faces_tuple, "face")

    ply_data = plyfile.PlyData([el_verts, el_faces])
    print("saving mesh to %s" % (ply_filename_out))
    ply_data.write(ply_filename_out)

    print(
        "converting to ply format and writing to file took {} s".format(
            time.time() - start_time
        )
    )



In [44]:
def reconstruct(
    decoder,
    num_iterations,
    latent_size,
    test_sdf,
    stat,
    clamp_dist,
    num_samples=8000,
    lr1=5e-4,
    l2reg=False,
):
    mul_d = 10
  #  decoder.train()
    if type(stat) == type(0.1):
        latent = torch.ones(1, latent_size).normal_(mean=0, std=stat).cuda()
    else:
        latent = torch.normal(stat[0].detach(), stat[1].detach()).cuda() #?
    #요것이 Latent
    latent.requires_grad = True
    #Latent Code Optimize!
    optimizer = torch.optim.Adam([latent], lr=lr1)

    loss_num = 0
    loss_l1 = torch.nn.L1Loss()

    decoder.eval()
    for e in range(num_iterations):
        sdf_data = unpack(test_sdf, num_samples).cuda()
        xyz = sdf_data[:, 0:3]
        sdf_gt = sdf_data[:, 3].unsqueeze(1)
        #SDF값
        sdf_gt = torch.clamp(sdf_gt, -clamp_dist, clamp_dist)
    #    print("sdf_gt:", min(sdf_gt), max(sdf_gt))
 
       # adjust_learning_rate(lr, optimizer, e, mul_d, adjust_lr_every)
        lr = lr1 * ((1 / mul_d) ** (num_iterations //  int(num_iterations / 2)))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        optimizer.zero_grad()

        latent_inputs = latent.expand(num_samples, -1)
        #latent와 xyz를 concat
        inputs = torch.cat([latent_inputs, xyz], 1).cuda()
        pred_sdf,_ = decoder(inputs)
        
        pred_sdf = torch.clamp(pred_sdf, -clamp_dist, clamp_dist)
    #    print("pred_sdf:", (pred_sdf))#, max(pred_sdf))
    
        #SDF 기준으로 loss값 설정
        loss = loss_l1(pred_sdf, sdf_gt)
        if l2reg:
            loss += 1e-4 * torch.mean(latent.pow(2))
        loss.backward()
        optimizer.step()
        loss_num = loss.item()

        if e % 50 == 0:
            print("num_iterations:",e)

            print("loss_num: ",loss_num)


    return loss_num, latent

In [45]:
#Model Load

# initialize Network
# If you couldn't install the easydict with pip/conda, please try this.
# git Clone https://github.com/makinacorpus/easydict.git

from easydict.easydict import EasyDict as edict 
#from easydict import EasyDict as edict 

lstm_args = edict()
lstm_args.steps = 8
lstm_args.latent_size = 256
lstm_args.h_size = 512
lstm_args.dim = [512, 64]

#lat_vecs = torch.nn.Embedding(len(sdf_loader), lstm_args.latent_size, max_norm=1)

mlp_args = edict()
mlp_args.layers = [256,256,256,256,256]
mlp_args.wn_layers = [0,1,2,3,4]
mlp_args.weight_norm = True
mlp_args.dropout_layers =[0,1,2,3,4]
mlp_args.dropout_prob = 0.05


device = "cuda" if torch.cuda.is_available() else "cpu"
weights = torch.load("./Pretrained Model/weights/weight/%s/weight_latest.pt"%class1, map_location=device)
decoder = DIT(mlp_args, lstm_args)
decoder.load_state_dict(weights)
#print(model)
decoder = torch.nn.DataParallel(decoder)
#print(weights)


In [46]:
iterations = 800
latent_size = 256
num_samples=8000
stat = 0.01
clamp_dist = 0.1
lr=5e-3
l2reg=True
resolution = 256
use_octree = False

for ii, npz in enumerate(files_npz_test):
#    print(npz)
    if "npz" not in npz:
        continue
    # files_dir_test #dirs
    print("Loading: ",npz)
#    DIRECTORY_MODELS
    npz1 = np.load(DIRECTORY_MODELS+ '/'+ npz)
    pos_tensor = torch.from_numpy(npz1["pos"])
    neg_tensor = torch.from_numpy(npz1["neg"])
    
    data_sdf = [pos_tensor, neg_tensor]

    for k in range(repeat):

        mesh_filename = os.path.join(mesh_dd, npz[:-4])
        latent_filename = os.path.join(lat_dd, npz[:-4] + ".pth")

        print("Reconstruction: ", npz)
        
        data_sdf[0] = data_sdf[0][torch.randperm(data_sdf[0].shape[0])]
        data_sdf[1] = data_sdf[1][torch.randperm(data_sdf[1].shape[0])]

        start = time.time()
        print("Latent_Filename: ", latent_filename)

        if not os.path.isfile(latent_filename):
            print("학습된 code를 활용하지 않는다는 의미!")
            loss_sum = 0
            err, latent = reconstruct(
                decoder,
                int(iterations),
                latent_size,
                data_sdf,
                0.01,  
                0.1,
                num_samples=8000,
                lr1=5e-3,
                l2reg=True,
            )
            loss_sum += err
            print("Current Error Avg: ", loss_sum/(ii+1))
            
        if not save_latvec_only:
            start = time.time()
            with torch.no_grad():
                #octree를 사용한다.
                if use_octree:
                    create_mesh_octree(
                        decoder, latent, mesh_filename, N=resolution, max_batch=int(2 ** 17), volume_size = 0.5, #volume_size = 0.1 > 그냥 네모가 작아질 뿐
                        clamp_func=clamping_function
                    )
                #Octree를 사용하지 않는다.
                else:
                    print('Octree를 사용하지 않고 만든다! 여기가 오래 걸리는 거 같은데?')
                    create_mesh(
                        decoder, latent, mesh_filename, N=resolution, max_batch=int(2 ** 17),
                    )                       


Loading:  ff2dbafa8d66856419fb4103277a6b93.npz
Reconstruction:  ff2dbafa8d66856419fb4103277a6b93.npz
Latent_Filename:  reconstruction/04256520/latent/ff2dbafa8d66856419fb4103277a6b93.pth
학습된 code를 활용하지 않는다는 의미!
num_iterations: 0
loss_num:  0.01542795542627573
num_iterations: 50
loss_num:  0.015230483375489712
num_iterations: 100
loss_num:  0.014616066589951515
num_iterations: 150
loss_num:  0.014517020434141159
num_iterations: 200
loss_num:  0.014156900346279144
num_iterations: 250
loss_num:  0.01359182596206665
num_iterations: 300
loss_num:  0.013824302703142166
num_iterations: 350
loss_num:  0.01258629560470581
num_iterations: 400
loss_num:  0.012398652732372284
num_iterations: 450
loss_num:  0.011590874753892422
num_iterations: 500
loss_num:  0.011607492342591286
num_iterations: 550
loss_num:  0.011182385496795177
num_iterations: 600
loss_num:  0.010983720421791077
num_iterations: 650
loss_num:  0.011154180392622948
num_iterations: 700
loss_num:  0.010473532602190971
num_iterations:

RuntimeError: Failed to decode SDF

In [14]:
files_npz_test

['10c7cdfdffe2243b88a89a28f04ce622.npz',
 '17ac3afd54143b797172a40a4ca640fe.npz',
 '17ad3ab1b1d12a7db26dc8ec64d633df.npz',
 '17c86b46990b54b65578b8865797aa0.npz',
 '18d123aaef6b911954eefcdc602d4520.npz',
 '18e86ba0172154f3bc0909d98a1ff2b4.npz',
 '1bba3fb413b93890947bbeb9022263b8.npz',
 '1beb0776148870d4c511571426f8b16d.npz',
 '1c93b0eb9c313f5d9a6e43b878d5b335.npz',
 '1d09583e9236b8d149d860a48be37092.npz',
 '1d0c128592385580e2129f6359ec27e3.npz',
 '1d5708929a4ae05842d1180c659735fe.npz',
 '1eb3af39d93c8bf2ddea2f4a0e0f0d2e.npz',
 '1fc2625479e798b11944f01d3ab2091b.npz',
 '24bdf389877fb7f21b1f694e36340ebb.npz',
 '24c2cc372c63603137678474be485ca.npz',
 '24d4c063f7a361bacbc6ff5546f4ec42.npz',
 '29b92db18649b64e959a8a7d5a8a5077.npz',
 '2a06adfb446c85c9f9d3f977c7090b2a.npz',
 '2a2caad9e540dcc687bf26680c510802.npz',
 '2a801b1918ef23f1121ca0b13e917b22.npz',
 '2af04ef09d49221b85e5214b0d6a7.npz',
 '2bc0d99cba39f7adbbf3143b1cb6076a.npz',
 '2c1f66380af03e4c5d1df55cbe0874aa.npz',
 '2d43c1430df8194ace5